In [1]:
import numpy
import pandas as pd
import math

In [2]:
def getESS(data, burninPerc=0, thin=None):
    """Effective sample size, as computed by BEAST Tracer, stolen from biopy via Gytis via someone else :D"""

    burnin = int(math.floor(burninPerc*len(data)/100.0))
    data = data[burnin:]

    if thin != None:
        data = data[0:len(data):max(len(data)/thin,1)]

    samples = len(data)

    assert len(data) > 1,"no stats for short sequences"

    maxLag = min(samples//3, 1000)

    gammaStat = [0,]*maxLag

    varStat = 0.0

    if type(data) != numpy.ndarray :
        data = numpy.array(data)

    normalizedData = data - data.mean()

    for lag in range(maxLag) :
        v1 = normalizedData[:samples-lag]
        v2 = normalizedData[lag:]
        v = v1 * v2
        gammaStat[lag] = sum(v) / len(v)

        if lag == 0 :
            varStat = gammaStat[0]
        elif lag % 2 == 0 :
            s = gammaStat[lag-1] + gammaStat[lag]
            if s > 0 :
                varStat += 2.0*s
            else:
                break

    if varStat > 0:
        return samples * gammaStat[0] / varStat
    else:
        return float("nan")

In [4]:
log_data = pd.read_csv('../../beast/pruned/discreteTraits/pruned_combined_10mBurnin.log', comment='#', sep="\t", index_col='state')	#parse the log file as a df


In [5]:
ESS_vals = log_data.apply(getESS)

In [7]:
ESS_vals.to_csv('../../beast/pruned/discreteTraits/results/ess.csv')